In [1]:
!nvidia-smi

Mon Dec 20 05:07:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:01:00.0 Off |                    0 |
| N/A   42C    P0    83W / 275W |  31015MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  Off  | 00000000:47:00.0 Off |                    0 |
| N/A   

In [1]:
!pip install torchio

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 164 kB 700 kB/s eta 0:00:01
     |████████████████████████████████| 48.4 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 9.9 MB/s eta 0:00:011
     |████████████████████████████████| 3.3 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 10.5 MB/s ta 0:00:011


In [3]:
!pip install monai

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 709 kB 221 kB/s eta 0:00:01


In [1]:
import random
import os
import os.path
import numpy as np
import math
import tqdm
import random
import copy
import torchio

import torch as t
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F 

import cv2
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter



In [2]:
t.backends.cudnn.enabled = True
#os.environ["CUDA_VISIBLE_DEVICES"]="1"
#device = t.device('cuda' if t.cuda.is_available() else 'cpu')

#device = t.device("cuda:2")

device = t.device("cuda:0")
device1 = t.device("cuda:0")
device2 = t.device("cuda:0")
print (device)
#device = "cuda:0"
#device1 = "cuda:0"
#device2 = "cuda:1"

train_path = "Data/train_data_tcia.pth"
test_path = "Data/test_data_tcia.pth"

file_name = "ResUNet_dice_log_tcia.txt"
file_name_1 = "ResUNet_loss_log_tcia.txt"
file_name_2 = "ResUNet_totalDice_log_tcia.txt"

epoch150_path_name = "Weights/ResUnet_tcia_42.pth"

y_pred_path = "OutputsTcia/ResUnetpred_tcia"
y_true_path = "OutputsTcia/ResUnettrue_tcia"


cuda:0


In [3]:
# ResUNet
# Encoder - Decoder Network

class ResnetBlock(nn.Module):

    def __init__(self, in_channels, out_channels,kernelSize=3):
        super().__init__()
        self.resnetblock1 = nn.Sequential(
            nn.Conv3d(in_channels,out_channels,kernel_size=kernelSize,padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=False)
            )
        self.resnetblock2 = nn.Sequential(
            nn.Conv3d(out_channels,out_channels,kernel_size=kernelSize,padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=False),
            nn.Conv3d(out_channels,out_channels,kernel_size=kernelSize,padding=1),
            nn.BatchNorm3d(out_channels)
            )
        #self.CBAM = CBAM(out_channels,16,['avg','max'])
        self.reluUnit = nn.Sequential(
            nn.ReLU(inplace=False)
            )

    def forward(self,x):
        x=self.resnetblock1(x)
        x1=x
        x=self.resnetblock2(x)
        #x=self.CBAM(x)
        x=x+x1
        x=self.reluUnit(x)
        return x
    
class Encoder(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        self.downConv  = nn.Sequential(
            nn.Conv3d(1,32,kernel_size=3,stride=2,padding=1,bias=False),
            nn.BatchNorm3d(32),
            nn.LeakyReLU(inplace=False),
        )

        self.layer21 = nn.Sequential(
            ResnetBlock(32,40),
            ResnetBlock(40,40)
        )

        self.layer22 = nn.Sequential(
            ResnetBlock(40,48),
            ResnetBlock(48,48)
        )

        self.layer23 = ResnetBlock(48,56)

    def forward(self,x):

        x0=self.downConv(x)
        x1=self.layer21(x0)
        x2=self.layer22(x1)
        x3=self.layer23(x2)
        #print(x3.shape)
        return x,x0,x1,x2,x3


class Decoder_part1(nn.Module):
    
    def __init__(self):
        super().__init__()
    
        self.layer23 = nn.Sequential(
            ResnetBlock(56,56),
            ResnetBlock(56,48)
        )
        self.layer22 = nn.Sequential(
            ResnetBlock(96,48),
            ResnetBlock(48,48),
            ResnetBlock(48,40)
        )
        self.layer21 = nn.Sequential(
            ResnetBlock(80,40),
            ResnetBlock(40,40),
            ResnetBlock(40,32)
        )
        self.layer20 = nn.Sequential(
            ResnetBlock(64,32),
            ResnetBlock(32,32)
        )
        self.transposeConv = nn.ConvTranspose3d(32,16,kernel_size=2,stride=2)
        self.layer10 = nn.Conv3d(17,16,kernel_size=3,padding=1)

    def forward(self,x,x0,x1,x2,x3):
        
        y=self.layer23(x3)
        y=t.cat([y,x2],dim=1)
        y=self.layer22(y)
        y=t.cat([y,x1],dim=1)
        y=self.layer21(y)
        y=t.cat([y,x0],dim=1)
        y=self.layer20(y)
        y=self.transposeConv(y)
        y=t.cat([y,x],dim=1)
        y=self.layer10(y)
        #print(y.shape)
        return y


class NetCore(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.encoder = Encoder()
        self.decoder_part1 = Decoder_part1()
  
    def forward(self,x):
        x,x0,x1,x2,x3=self.encoder(x)
        y=self.decoder_part1(x,x0,x1,x2,x3)
        return y


class ResUNetModel(nn.Module):
  
    def __init__(self):
        super().__init__()
        self.core = NetCore()
        self.finalConv = nn.Sequential(
            nn.Conv3d(16,8,1),
            nn.Softmax(dim=1),
        ).cuda()
  
    def forward(self,x):
        y=self.core(x)
        y=self.finalConv(y)
        #print(y.shape)
        return y
    

In [4]:
#dataloading
class HaN_Dataset(Dataset):
    
    def __init__(self, root_dir=None, transform=False, alpha=1000, sigma=30, alpha_affine=0.04):
        super().__init__()
        self.path = root_dir
        self.datas = t.load(self.path)
        
        self.transform = transform
        self.alpha = alpha
        self.sigma = sigma
        self.alpha_affine = alpha_affine
    
    def __getitem__(self, index):
        data = self.datas[index]
        img = data['img'].numpy().astype(np.float32)
        
        if not self.transform:
            masklst = []
            for mask in data['mask']:
                if mask is None:
                    mask = np.zeros((1,img.shape[0],img.shape[1],img.shape[2])).astype(np.uint8)
                masklst.append(mask.astype(np.uint8).reshape((1,img.shape[0],img.shape[1],img.shape[2]))) 
            mask0 = np.zeros_like(masklst[0]).astype(np.uint8)
            for mask in masklst:
                mask0 = np.logical_or(mask0, mask).astype(np.uint8)
            mask0 = 1 - mask0
            return t.from_numpy(img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))), t.from_numpy(np.concatenate([mask0]+masklst, axis=0)), True
        
        im_merge = np.concatenate([img[...,None]]+[mask.astype(np.float32)[...,None] for mask in data['mask']], axis=3)
        # Apply transformation on image
        im_merge_t, new_img = self.elastic_transform3Dv2(im_merge,self.alpha,self.sigma,min(im_merge.shape[1:-1])*self.alpha_affine)
        # Split image and mask ::2, ::2, ::2
        im_t = im_merge_t[...,0]
        im_mask_t = im_merge_t[..., 1:].astype(np.uint8).transpose(3, 0, 1, 2)
        mask0 = np.zeros_like(im_mask_t[0, :, :, :]).reshape((1,)+im_mask_t.shape[1:]).astype(np.uint8)
        im_mask_t_lst = []
        flagvect = np.ones((8,), np.float32)
        retflag = True
        for i in range(7):
            im_mask_t_lst.append(im_mask_t[i,:,:,:].reshape((1,)+im_mask_t.shape[1:]))
            if im_mask_t[i,:,:,:].max() != 1: 
                retflag = False
                flagvect[i+1] = 0
            mask0 = np.logical_or(mask0, im_mask_t[i,:,:,:]).astype(np.uint8)
        if not retflag: flagvect[0] = 0
        mask0 = 1 - mask0
        return t.from_numpy(im_t.reshape((1,)+im_t.shape[:3])), t.from_numpy(np.concatenate([mask0]+im_mask_t_lst, axis=0)), flagvect
        
    def __len__(self):
        return len(self.datas)
    
    def elastic_transform3Dv2(self, image, alpha, sigma, alpha_affine, random_state=None):
        """Elastic deformation of images as described in [Simard2003]_ (with modifications).
        .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
             Convolutional Neural Networks applied to Visual Document Analysis", in
             Proc. of the International Conference on Document Analysis and
             Recognition, 2003.
         Based on https://gist.github.com/erniejunior/601cdf56d2b424757de5
         From https://www.kaggle.com/bguberfain/elastic-transform-for-data-augmentation
        """
        # affine and deformation must be slice by slice and fixed for slices
        if random_state is None:
            random_state = np.random.RandomState(None)
        shape = image.shape # image is contatenated, the first channel [:,:,:,0] is the image, the second channel 
        # [:,:,:,1] is the mask. The two channel are under the same tranformation.
        shape_size = shape[:-1] # z y x
        # Random affine
        shape_size_aff = shape[1:-1] # y x
        center_square = np.float32(shape_size_aff) // 2
        square_size = min(shape_size_aff) // 3
        pts1 = np.float32([center_square + square_size, [center_square[0]+square_size, center_square[1]-square_size], center_square - square_size])
        pts2 = pts1 + random_state.uniform(-alpha_affine, alpha_affine, size=pts1.shape).astype(np.float32)
        M = cv2.getAffineTransform(pts1, pts2)
        new_img = np.zeros_like(image)
        for i in range(shape[0]):
            new_img[i,:,:,0] = cv2.warpAffine(image[i,:,:,0], M, shape_size_aff[::-1], borderMode=cv2.BORDER_CONSTANT, borderValue=0.)
            for j in range(1, 8):
                new_img[i,:,:,j] = cv2.warpAffine(image[i,:,:,j], M, shape_size_aff[::-1], flags=cv2.INTER_NEAREST, borderMode=cv2.BORDER_TRANSPARENT, borderValue=0)
        dx = gaussian_filter((random_state.rand(*shape[1:-1]) * 2 - 1), sigma) * alpha
        dy = gaussian_filter((random_state.rand(*shape[1:-1]) * 2 - 1), sigma) * alpha
        x, y = np.meshgrid(np.arange(shape_size_aff[1]), np.arange(shape_size_aff[0]))
        indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1))
        new_img2 = np.zeros_like(image)
        for i in range(shape[0]):
            new_img2[i,:,:,0] = map_coordinates(new_img[i,:,:,0], indices, order=1, mode='constant').reshape(shape[1:-1])
            for j in range(1, 8):
                new_img2[i,:,:,j] = map_coordinates(new_img[i,:,:,j], indices, order=0, mode='constant').reshape(shape[1:-1])
        return np.array(new_img2), new_img
# %%


traindataset = HaN_Dataset(train_path, transform=True)
traindataloader = DataLoader(traindataset, batch_size=1, shuffle=True)
testdataset = HaN_Dataset(test_path, transform=False)
testdataloader = DataLoader(testdataset, batch_size=1)

print(len(traindataloader),len(testdataloader))


211 20


In [5]:
# loss functions
def crossentropy(y_pred, y_true, flagvec, device):
    retv = - t.sum(t.sum(t.sum(t.sum(t.log(t.clamp(y_pred,1e-6,1))*y_true.type(t.cuda.FloatTensor),4),3),2),0) * flagvec.to(device)
    return t.sum(retv / (t.sum(t.sum(t.sum(t.sum(y_true.type(t.cuda.FloatTensor),4),3),2),0) + 1e-6)) / y_true.size()[1]

# %%

def tversky_loss_wmask(y_pred, y_true, flagvec, device):
    alpha = 0.5
    beta  = 0.5
    ones = t.ones_like(y_pred) 

    p0 = y_pred      # proba that voxels are class i
    p1 = ones-y_pred # proba that voxels are not class i
    g0 = y_true.type(t.cuda.FloatTensor)
    g1 = ones-g0
    num = t.sum(t.sum(t.sum(t.sum(p0*g0, 4),3),2),0) #(0,2,3,4)) #K.sum(p0*g0, (0,1,2,3))
    den = num + alpha*t.sum(t.sum(t.sum(t.sum(p0*g1,4),3),2),0) + beta*t.sum(t.sum(t.sum(t.sum(p1*g0,4),3),2),0) #(0,2,3,4))

    T = t.sum((num* flagvec.to(device))/(den+1e-5))
    return t.sum(flagvec.to(device))-T

# %%

def focal(y_pred, y_true, flagvec, device):
    retv = - t.mean(t.mean(t.mean(t.mean(t.log(t.clamp(y_pred,1e-6,1))*y_true.type(t.cuda.FloatTensor)*t.pow(1-y_pred,2),4),3),2),0) * flagvec.to(device)
    return t.sum(retv)

In [6]:
#DSC evaluation
def caldice(y_pred, y_true):

    y_pred = y_pred.data.cpu().numpy().transpose(1,0,2,3,4) # inference should be arg max
    y_pred = np.argmax(y_pred, axis=0).squeeze() # z y x
    y_true = y_true.data.cpu().numpy().transpose(1,0,2,3,4).squeeze() # .cpu()
    avgdice = []
    y_pred_1 = y_pred==1
    y_true_1 = y_true[1,:,:,:]
    if y_pred_1.sum() + y_true_1.sum() == 0: avgdice.append(-1)
    else: avgdice.append(2.*(np.logical_and(y_pred_1, y_true_1).sum()) / (1.0*(y_pred_1.sum() + y_true_1.sum())))
    
    y_pred_1 = y_pred==2
    y_true_1 = y_true[2,:,:,:]
    if y_pred_1.sum() + y_true_1.sum() == 0: avgdice.append(-1)
    else: avgdice.append(2.*(np.logical_and(y_pred_1, y_true_1).sum()) / (1.0*(y_pred_1.sum() + y_true_1.sum())))
    
    y_pred_1 = y_pred==3
    y_true_1 = y_true[3,:,:,:]
    if y_pred_1.sum() + y_true_1.sum() == 0: avgdice.append(-1)
    else: avgdice.append(2.*(np.logical_and(y_pred_1, y_true_1).sum()) / (1.0*(y_pred_1.sum() + y_true_1.sum())))
    
    y_pred_1 = y_pred==4
    y_true_1 = y_true[4,:,:,:]
    if y_pred_1.sum() + y_true_1.sum() == 0: avgdice.append(-1)
    else: avgdice.append(2.*(np.logical_and(y_pred_1, y_true_1).sum()) / (1.0*(y_pred_1.sum() + y_true_1.sum())))
    
    y_pred_1 = y_pred==5
    y_true_1 = y_true[5,:,:,:]
    if y_pred_1.sum() + y_true_1.sum() == 0: avgdice.append(-1)
    else: avgdice.append(2.*(np.logical_and(y_pred_1, y_true_1).sum()) / (1.0*(y_pred_1.sum() + y_true_1.sum())))
    
    y_pred_1 = y_pred==6
    y_true_1 = y_true[6,:,:,:]
    if y_pred_1.sum() + y_true_1.sum() == 0: avgdice.append(-1)
    else: avgdice.append(2.*(np.logical_and(y_pred_1, y_true_1).sum()) / (1.0*(y_pred_1.sum() + y_true_1.sum())))
    
    y_pred_1 = y_pred==7
    y_true_1 = y_true[7,:,:,:]
    if y_pred_1.sum() + y_true_1.sum() == 0: avgdice.append(-1)
    else: avgdice.append(2.*(np.logical_and(y_pred_1, y_true_1).sum()) / (1.0*(y_pred_1.sum() + y_true_1.sum())))

    for dice in avgdice: 
        if dice != -1:
            assert 0 <= dice <= 1
    return avgdice

def get_yPredDash(y_pred):
    
    y_pred = y_pred.data.cpu().numpy() # inference should be arg max
    y_pred_1 = np.zeros_like(y_pred).astype(np.uint8)
    y_pred = np.argmax(y_pred, axis=1).squeeze() # z y x
    
    for i in range(8):
        y_pred_1[:,i,:,:,:] = y_pred==i
    
    return y_pred_1



In [10]:
#ResUnet
file_name = "ResUNet_dice_log_tcia.txt"
file_name_1 = "ResUNet_loss_log_tcia.txt"
file_name_2 = "ResUNet_totalDice_log_tcia.txt"
epoch150_path_name = "Weights/ResUNet_epoch150_path"
epoch150_50_path_name = "Weights/ResUNet_epoch150_50_path"


UNet_Model = ResUNetModel()
print("ResUNetmodel created")

ResUNetmodel created


In [11]:
#ResUnet  training
epoch1=150
epoch2=50
UNet_Model = ResUNetModel()
model=UNet_Model.to(device)

lossweight = np.array([2.22, 1.06, 1.02, 1.74, 1.93, 1.93, 1.13, 1.15], np.float32)
ceweight = 0.05
focweight = 0.5

optimizer = t.optim.RMSprop(model.parameters(),lr = 2e-3)

#for epoch in range(anatomy_epoch1):
for epoch in range(epoch1):
    i=0
    j=0
    train_loss = 0
    print("epoch ", epoch+1)
    for x_train, y_train, flagvec in tqdm.tqdm(traindataloader):
        
        i+=1
        model.train()
        #print(f'epoch {epoch+1} batch {i} tensor_shape {x_train.shape}')
        if x_train.shape[-2] > 300:
            x_train = x_train[...,10:,:]
            y_train = y_train[...,10:,:]
        
        x_train = x_train.to(device1)
        out =model(x_train)
        
        out = out.to(device2)
        y_train = y_train.to(device2)
        loss = tversky_loss_wmask(out, y_train, flagvec*t.from_numpy(lossweight), device2)
        celoss = crossentropy(out, y_train, flagvec*t.from_numpy(lossweight), device2)
        floss = focal(out, y_train, flagvec*t.from_numpy(lossweight), device2)
        optimizer.zero_grad()
        (loss + (ceweight*celoss) + (focweight*floss)).backward()
        optimizer.step()
        train_loss += loss.item() + (ceweight*celoss).item() + (focweight*floss).item()

        y_train.detach_()
        out.detach_()
        del loss, x_train, y_train, out, floss, celoss
        
    print("train loss: ",train_loss/(len(traindataloader)-j),'\n')
    
    testtq = tqdm.tqdm(testdataloader)#, desc='loss', leave=True)
    test_loss = 0
    testloss = [0 for _ in range(7)]
    for x_test, y_test, flagvecTest in testtq:
        with t.no_grad():
            model.eval()
            x_test = x_test.to(device1)
            y_test = y_test.to(device2)
            out = model(x_test)
            out = out.to(device2)
            loss = tversky_loss_wmask(out, y_test, flagvecTest*t.from_numpy(lossweight), device2)
            celoss = crossentropy(out, y_test, flagvecTest*t.from_numpy(lossweight), device2)
            floss = focal(out, y_test, flagvecTest*t.from_numpy(lossweight), device2)
            test_loss += loss.item() + (ceweight*celoss).item() + (focweight*floss).item()
            dice= caldice(out, y_test)
            testtq.set_description("dice value %f" % (sum(dice)/7))
            testtq.refresh()
            testloss = [l+tl if l != -1 else tl for l,tl in zip(dice, testloss)]
            del out, x_test, y_test
            
    print(f"test loss: {test_loss/len(testdataloader)}")
    
    with open(file_name_1, "a") as f:
        f.write(f"epoch {epoch+1}")
        f.write(f"\ttrain loss: {train_loss/(len(traindataloader)-j)}")
        f.write(f"\ttest loss: {test_loss/len(testdataloader)}\n")
                           
    testloss = [l / len(testtq) for l in testloss]
    with open(file_name_2, "a") as f:
        f.write(f"epoch {epoch+1}")
        f.write('\tDice coeff %.4f, %.4f, %.4f, %.4f, %.4f, %.4f, %.4f \n' % tuple(testloss))
            
    if (epoch+1)%10==0:
        
        t.save({
        'epoch': epoch+1,
        'model_state_dict':  model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
        }, epoch150_path_name)
        
        with open(file_name, "a") as f:
            f.write(f"epoch {epoch+1}")
            f.write('\tDice coeff %.4f, %.4f, %.4f, %.4f, %.4f, %.4f, %.4f \n' % tuple(testloss))
        
        for x,y,_ in testdataloader:
            with t.no_grad():
                y_pred = model(x.to(device1))
                y_predDash = get_yPredDash(y_pred)
                np.save(y_pred_path,y_predDash)
                np.save(y_true_path,y.cpu())
                #print(y_pred.shape, y.shape) 
                break

# %%


epoch  1


100%|█████████████████████████████████████████| 211/211 [10:13<00:00,  2.91s/it]


train loss:  8.019301193105873 



dice value 0.005945: 100%|██████████████████████| 20/20 [00:11<00:00,  1.68it/s]


test loss: 10.471872038021683
epoch  2


...

train loss:  1.8337857894969374 



dice value 0.790461: 100%|██████████████████████| 20/20 [00:11<00:00,  1.69it/s]


test loss: 2.3373431260697544
epoch  43


 33%|█████████████▉                            | 70/211 [03:27<06:57,  2.96s/it]


KeyboardInterrupt: 

In [19]:
#evaluation Step1
weight_path= "Weights/ResUnet_tcia_42.pth"
UNet_Model =  ResUNetModel()
loaded_model = t.load(weight_path, map_location='cuda:0')
print(loaded_model["epoch"])
UNet_Model.load_state_dict(loaded_model["model_state_dict"])
model=UNet_Model.to(device)

testdataset = HaN_Dataset(test_path, transform=False)
testdataloader = DataLoader(testdataset, batch_size=1)
print(len(testdataloader))

42
20


In [21]:
#evaluation Step2
testloss = [0 for _ in range(7)]
testtq = tqdm.tqdm(testdataloader, desc='loss', leave=True)
   
for x_test, y_test, _ in testtq:
    with t.no_grad():
        model.eval()
        x_test = x_test.to(device1) 
        y_test = y_test.to(device2) 
        o = model(x_test)
        o = o.to(device2) 
        loss = caldice(o, y_test)
        testtq.set_description("test loss %f" % (sum(loss)/7))
        testtq.refresh() 
        testloss = [l+tl if l != -1 else tl for l,tl in zip(loss, testloss)]  
        del x_test, y_test, o
        
testloss = [l / len(testtq) for l in testloss]
print('\tDice coeff %.4f, %.4f, %.4f, %.4f, %.4f, %.4f, %.4f \n' % tuple(testloss))

test loss 0.769712: 100%|███████████████████████| 20/20 [00:11<00:00,  1.79it/s]

	Dice coeff 0.8209, 0.5709, 0.9189, 0.6950, 0.7416, 0.8063, 0.8432 

